# Introduction
This project analyzes NYC campaign contribution data, covering over 1.4 million contributions.  
The goal is to understand donor behavior, detect patterns in donation amounts, geographies, and  
across time. Before cleaning, I conduct a exploration to guide   
data transformations/cleaning.  

In [198]:
import pandas as pd
import os

data_path = os.path.join("..", "data", "Campaign_Contributions.csv")

try:
    campaign_df = pd.read_csv(data_path, low_memory = False)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Data file not found. Please download 'Campaign_Contributions.csv' and place it in the /data folder.")

Data loaded successfully.


# Dataset Overview

The dataset contains around 1.4 million rows and 54 columns. Most columns are strings (`object` type), with a small number of numeric and date fields.

- Shape: (1,408,814, 54)  
- Data types:  
  - 45 object  
  - 8 float64  
  - 1 int64

### Columns for Analysis

These columns are well-populated and useful for analysis:

- Donation Info: `AMNT`, `MATCHAMNT`, `PREVAMNT`, `DATE`
- Contributor Info: `NAME`, `CITY`, `STATE`, `ZIP`, `OCCUPATION`, `EMPNAME`
- Candidate/Campaign Info: `RECIPNAME`, `ELECTION`, `OFFICECD`, `CANCLASS`
- Other kept Fields: `PAY_METHOD`, `C_CODE`

### Columns Dropped
These have too many missing values or are redundant:
- Intermediary Details:  
  `INTERMNAME`, `INTERMNO`, `INTSTRNO`, `INTSTRNM`, `INTAPTNO`, `INTCITY`, `INTST`, `INTZIP`, `INTEMPNAME`, `INTEMPSTNO`, `INTEMPSTNM`, `INTEMPCITY`, `INTEMPST`, `INTOCCUPA`, `INT_C_CODE`  

- sparse and too granular Address Fragments:  
  `STRNO`, `STRNAME`, `APARTMENT`, `EMPSTRNO`, `EMPSTRNAME`, `EMPCITY`, `EMPSTATE`  

- Sparse and Irrelevant Columns:  
  `PAGENO`, `SEQUENCENO`, `REFUNDDATE`, `FILING`, `SCHEDULE`, `REFNO`, `PURPOSECD`, `EXEMPTCD`, `ADJTYPECD`, `RR_IND`, `SEG_IND`, `COMMITTEE`  

- Redundant Name Splits:  
  `CANDFIRST`, `CANDMI`  

Over 20 columns have more than 95% missing values, so im focusing on the cleaner, more useful subset for the project.

### Date Range Consideration

The data includes entries as early as the 1920s due to formatting errors or legacy input.  
To ensure modern relevance, cleaner comparisons, and consistent naming convention for `RECIPNAME` im are limiting the dataset to records from 2007 onward.

In [190]:
from IPython.display import display
pd.set_option('display.max_columns', None)

# Show number of rows and columns
print("data shape (rows, columns):")
display(campaign_df.shape)

# Show count of data types
print("data type counts:")
display(campaign_df.dtypes.value_counts())

# Compute and sort by missing value counts
missing_counts = campaign_df.isna().sum().sort_values(ascending=False)

# Show top 20 columns by highest ratio of missing data
print("top 20 columns by missing value ratio:")
display(campaign_df.isna().mean().sort_values(ascending=False).head(20))

# Show top 20 columns by lowest ratio of missing data
print("top 20 columns by lowest missing value ratio:")
display(campaign_df.isna().mean().sort_values(ascending=True).head(20))

data shape (rows, columns):


(1408814, 54)

data type counts:


object     45
float64     8
int64       1
Name: count, dtype: int64

top 20 columns by missing value ratio:


PAGENO        1.000000
SEQUENCENO    1.000000
INTAPTNO      0.999889
EXEMPTCD      0.999878
INTSTRNO      0.999607
INTSTRNM      0.999606
APARTMENT     0.997056
PURPOSECD     0.995758
STRNO         0.983874
STRNAME       0.983350
REFUNDDATE    0.982065
ADJTYPECD     0.982056
INTEMPSTNO    0.967557
INTEMPSTNM    0.967292
INTEMPST      0.964521
INTEMPCITY    0.964477
INTEMPNAME    0.963229
INTOCCUPA     0.962113
INTZIP        0.957769
INTST         0.957605
dtype: float64

top 20 columns by lowest missing value ratio:


ELECTION      0.000000e+00
OFFICECD      0.000000e+00
RECIPID       0.000000e+00
RECIPNAME     0.000000e+00
FILING        0.000000e+00
SCHEDULE      0.000000e+00
REFNO         0.000000e+00
C_CODE        7.098169e-07
PAY_METHOD    7.098169e-07
AMNT          7.098169e-07
PREVAMNT      7.098169e-07
MATCHAMNT     7.098169e-07
NAME          1.916506e-05
DATE          5.678535e-04
COMMITTEE     1.025685e-03
CANCLASS      1.025685e-03
RR_IND        1.026395e-03
CITY          4.580449e-03
STATE         5.144753e-03
SEG_IND       5.218574e-03
dtype: float64

# Data Cleaning

## Dropping Irrelevant and Very Spare Columns

In [191]:
keep_columns = [
    'DATE', 'AMNT', 'MATCHAMNT', 'PREVAMNT',
    'NAME', 'CITY', 'STATE', 'ZIP', 'OCCUPATION', 'EMPNAME',
    'RECIPNAME', 'ELECTION', 'OFFICECD', 'CANCLASS',
    'PAY_METHOD', 'C_CODE'
]

campaign_df = campaign_df[keep_columns].copy()
campaign_df.columns

Index(['DATE', 'AMNT', 'MATCHAMNT', 'PREVAMNT', 'NAME', 'CITY', 'STATE', 'ZIP',
       'OCCUPATION', 'EMPNAME', 'RECIPNAME', 'ELECTION', 'OFFICECD',
       'CANCLASS', 'PAY_METHOD', 'C_CODE'],
      dtype='object')

## Dropping Duplicate Rows

In [192]:
# delete duplicate rows
campaign_df = campaign_df.drop_duplicates()

## Date Parsing 
- Converted the `DATE` and `REFUNDDATE` columns to datetime format.
- `DATE` has only 801 unparsable entries (<.1%), which can be discarded.
- `REFUNDDATE` is mostly null (>95%), which aligns with the fact that refunds are rare.
- Created `YEAR` and `MONTH` features from `DATE` to allow for quick monthly and yearly splits. 
- Dropped entries before 2007

In [193]:
# Convert DATE and REFUNDDATE to datetime
campaign_df['DATE'] = pd.to_datetime(campaign_df['DATE'], errors='coerce')

# Drop Nan and entries from before 2007
campaign_df = campaign_df[campaign_df['DATE'].dt.year >= 2007]

# Check how many dates cant be parsed
invalid_dates = campaign_df['DATE'].isna().sum()

print(f"Invalid DATE entries: {invalid_dates}")

# Create YEAR and MONTH fields
campaign_df['YEAR'] = campaign_df['DATE'].dt.year
campaign_df['MONTH'] = campaign_df['DATE'].dt.month

Invalid DATE entries: 0


# Dropping Negative Donations

In [194]:
donation_cols = ['AMNT', 'MATCHAMNT', 'PREVAMNT']

for col in donation_cols:
    # Count how many are non-positive
    invalid_count = (campaign_df[col] <= 0).sum()
    print(f"{col}: {invalid_count} rows with zero or negative values")

    # Keep only rows where the value is strictly greater than 0
campaign_df = campaign_df[campaign_df['AMNT'] > 0]

AMNT: 19955 rows with zero or negative values
MATCHAMNT: 407626 rows with zero or negative values
PREVAMNT: 823865 rows with zero or negative values


## String Cleaning and Standardization

To ensure consistency across categorical text fields, I apply basic string standardization.  
This includes converting values to uppercase, stripping leading/trailing whitespace, and casting to string.

This process is important for columns used in grouping or filtering, as inconsistent formatting can lead to inaccurate aggregation.

I apply this cleaning step only to clearly structured fields.

Columns cleaned:
- `NAME`
- `CITY`
- `STATE`
- `ZIP`
- `OCCUPATION`
- `EMPNAME`
- `RECIPNAME`

Zip codes are shortened to the first 5 decimals for consistency, and states are forced into standard state codes.


In [195]:
# Strip whitespace and standardize case
contributor_cols = ['NAME', 'CITY', 'STATE', 'ZIP', 'OCCUPATION', 'EMPNAME', 'RECIPNAME']
for col in contributor_cols:
    campaign_df[col] = campaign_df[col].astype(str).str.strip().str.upper()

# Replace empty strings or placeholders with NaN
campaign_df[contributor_cols] = campaign_df[contributor_cols].replace(['', 'NAN', 'NULL'], pd.NA)

# Clean ZIP codes: keep only first 5 digits and ignore extensions
campaign_df['ZIP'] = campaign_df['ZIP'].str.extract(r'(\d{5})')

# Remove entries with invalid state codes
valid_states = set([
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID',
    'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC',
    'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD',
    'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'
])
campaign_df.loc[~campaign_df['STATE'].isin(valid_states), 'STATE'] = pd.NA

## Categorical/Encoded Cleaning

The following additional fields were cleaned and standardized for analysis:

- `ELECTION`: Extracted the 4-digit election year and cast to integer. This simplifies grouping and trend analysis.
- `CANCLASS`: Standardized campaign finance classifications and removed invalid entries.
- `OFFICECD`: Mapped numeric codes to office labels for better readability.
- `C_CODE`: Normalized contributor types and filtered to valid codes.
- `PAY_METHOD`: Converted numerical codes to readable payment method labels, stored in a separate column `PAY_METHOD_LABEL`.

These transforms ensure consistency across categorical/encoded features and allow for more meaningful aggregation and visualization.


In [196]:
# Extract 4-digit year from ELECTION column
campaign_df['ELECTION'] = campaign_df['ELECTION'].astype(str).str.extract(r'(\d{4})')
campaign_df['ELECTION'] = campaign_df['ELECTION'].astype('Int64') 

# Verify CANCLASS is valid 
campaign_df['CANCLASS'] = campaign_df['CANCLASS'].str.strip().str.upper()
valid_classes = ['PARTICIPANT', 'NON-PARTICIPANT', 'LIMITED PARTICIPANT', 'UNDETERMINED']
campaign_df.loc[~campaign_df['CANCLASS'].isin(valid_classes), 'CANCLASS'] = pd.NA

# Verify office codes are in range and map to string
office_map = {
    '1': 'Mayor', '2': 'Public Advocate', '3': 'Comptroller',
    '4': 'Borough President', '5': 'City Council', '6': 'Undeclared'
}
campaign_df['OFFICECD'] = campaign_df['OFFICECD'].astype(str).str.strip()
campaign_df = campaign_df[campaign_df['OFFICECD'].isin(office_map.keys())]
campaign_df['OFFICECD'] = campaign_df['OFFICECD'].map(office_map)

# Verify company codes are in the valid code set
campaign_df['C_CODE'] = campaign_df['C_CODE'].str.strip().str.upper()
valid_c_codes = ['CAN', 'CORP', 'EMPO', 'FAM', 'IND', 'LLC', 'OTHR', 'PART', 'PCOMC', 'PCOMP', 'PCOMZ', 'SPO', 'UNKN']
campaign_df.loc[~campaign_df['C_CODE'].isin(valid_c_codes), 'C_CODE'] = pd.NA

# Map integer payment methods to String
pay_method_map = {
    0: 'Unknown', 1: 'Cash', 2: 'Check', 3: 'Other', 4: 'Credit Card', 5: 'Money Order'
}
campaign_df['PAY_METHOD'] = campaign_df['PAY_METHOD'].astype('Int64') 
campaign_df['PAY_METHOD_LABEL'] = campaign_df['PAY_METHOD'].map(pay_method_map)

OFFICECD
5     305850
1     167547
11    165500
55    155773
4      49541
44     48861
33     39877
2      37061
3      34876
6       9587
22      5903
IS      1392
66       406
Name: count, dtype: int64

# Save to CSV

In [180]:
# Save cleaned data to CSV
campaign_df.to_csv("../data/Cleaned_Donor_Data.csv", index=False)